# Prepare mangrove centroids
Author: Sarah Hülsen

Code to replicate main results of Mangroves and their services are at risk from tropical cyclones and sea level rise under climate change.
This notebook contains code to prepare Global Mangrove Watch (Bunting et al., 2018) global mangrove extents and rates of sea level rise data (Garner et al., 2021) for further analysis.
Mangrove data are prepared following these steps:
* calculate centroids of mangrove polygons (GMW)
* extract RSLR rates from raw NetCDF files
* merge RSLR data with mangrove centroids
* implement 3 intensity categories

In [ ]:
from pathlib import Path
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import box

# define data paths 
data_dir = Path("../../data")

In [ ]:
# define data to be analysed
hab = 'mangroves'
source = 'Bunting'
year = '2020'
tc_basins = ['AP', 'IO', 'SH', 'WP']
cols = ['geometry', 'area', 'longitude', 'latitude']
ssps = [245, 370, 585]

## Create mangrove centroids

In [ ]:
# read data
data = gpd.read_file(data_dir.joinpath('mangroves', 'Bunting_mangroves', 'gmw_v3_2020_vec', 'gmw_v3_2020_vec.shp'))

# calculate polygon areas
proj_eck4 = '+proj=eck4 +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs'  # Eckert IV projection
data = data.to_crs(proj_eck4) # project layer to equal area projection
data['area'] = data['geometry'].area

# extract polygon centroids
data_cents = data.copy()
data_cents.geometry = data_cents['geometry'].centroid
data_cents = data_cents.to_crs(4326) # crs back to WGS84

# save as csv for further analysis
data_cents['longitude'] = data_cents['geometry'].x
data_cents['latitude'] = data_cents['geometry'].y
data_cents = data_cents[cols]
data_cents.to_csv(data_dir.joinpath(f'{source}_{hab}_{year}_global.csv'), index=False)

## Regional Relative Sea Level Rise data

In [ ]:
file_name = f'total_ssp245_medium_confidence_rates.nc'
dataset = xr.open_dataset(data_dir.joinpath(file_name))
dataset

<xarray.Dataset> Size: 397MB
Dimensions:                (locations: 66190, years: 14, quantiles: 107)
Coordinates:
  * locations              (locations) int32 265kB 1 2 ... 1018003580 1018003590
  * years                  (years) int32 56B 2020 2030 2040 ... 2130 2140 2150
  * quantiles              (quantiles) float64 856B 0.0 0.001 ... 0.999 1.0
Data variables:
    lat                    (locations) float32 265kB ...
    lon                    (locations) float32 265kB ...
    sea_level_change_rate  (quantiles, years, locations) float32 397MB ...
Attributes:
    description:  Combined confidence output file for AR6 sea-level change pr...
    history:      Created Wed Jul 14 14:08:01 2021
    source:       Files Combined: /projects/kopp/ar6/regional/pboxes_rates/pb...

## Extract regional RSLR rates from NetCDFs

In [ ]:
# new version
# extract median rates of regional relative sea level rise until 2100
df = pd.DataFrame()
start_year = 2020
end_year = 2100

for ssp in ssps:
    file_name = f'total_ssp{ssp}_medium_confidence_rates.nc'
    dataset = xr.open_dataset(data_dir.joinpath(file_name))

    # extract sea level change rates variable
    sea_level_change = dataset['sea_level_change_rate']

    # fix quantile dimension at median
    sea_level_change_at_0_5 = sea_level_change.sel(quantiles=0.5)

    # slice years 2020 - 2100
    sea_level_change_filtered = sea_level_change_at_0_5.sel(years=slice(start_year, end_year))

    # calculate the median across the filtered years
    median_values = sea_level_change_filtered.median(dim='years')

    # values to dataframe
    df[f'median_rslr_{ssp}'] = median_values.values

    dataset.close()

# add location, latitude, and longitude information to the DataFrame
df['location'] = median_values['locations'].values
df['latitude'] = dataset['lat'].sel(locations=median_values['locations']).values
df['longitude'] = dataset['lon'].sel(locations=median_values['locations']).values

# Save the DataFrame to a CSV file
df.to_csv(data_dir.joinpath('rslr_all_ssps_locations_2020-2100.csv'), index=False)

### Match regional RSLR data with mangrove extents

In [ ]:
# new
source = 'Bunting'
# read rslr data and convert to geodataframe
df = pd.read_csv(data_dir.joinpath('rslr_all_ssps_locations.csv'))
gdf = gpd.GeoDataFrame(data=df, geometry=gpd.points_from_xy(df["longitude"], df["latitude"]), crs="EPSG:4326")
gdf = gdf.drop(columns=['latitude', 'longitude', 'location'])

# load ecosystem data
exp = pd.read_csv(data_dir.joinpath('mangroves', f'{source}_mangroves_2020_global.csv'))
exp_gdf = gpd.GeoDataFrame(data=exp, geometry=gpd.points_from_xy(exp["longitude"], exp["latitude"]), crs="EPSG:4326")
exp_bounds = exp_gdf.total_bounds

# clip rslr data to ecosystem bounds
bounds_buffered = box(*exp_bounds).buffer(2) # add 2 degree buffer to the bounds
buffered_gdf = gpd.GeoDataFrame(geometry=[bounds_buffered], crs=exp_gdf.crs)
gdf_clip = gpd.clip(gdf, buffered_gdf)

# reproject to equal area (Eckert IV)
proj_eck4 = '+proj=eck4 +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs'  # Eckert IV projection
gdf_clip = gdf_clip.to_crs(proj_eck4)
exp_gdf = exp_gdf.to_crs(proj_eck4)

# match points
join = gpd.sjoin_nearest(exp_gdf, gdf_clip, distance_col="rslr_distance")
join = join.drop(columns=['index_right'])

# reproject
join = join.to_crs("EPSG:4326")
join = join.drop_duplicates(keep='first')

# sort into intensity categories
categories = [(0, 4), (4, 7), (7, np.inf)]

for ssp in ssps:
    for i, category in enumerate(categories):
        lower_bound, upper_bound = category
        cat = i+1
        label = f'RSLR{cat}_ssp{ssp}'
        join[label] = np.where(
            (join[f'median_rslr_{ssp}'] >= lower_bound) & (join[f'median_rslr_{ssp}'] < upper_bound),
            1,
            0
        )

    # adjust categories to make them exclusive (always prioritize highest possible)
    join[f'RSLR2_ssp{ssp}'] = join[f'RSLR2_ssp{ssp}'] & ~join[f'RSLR3_ssp{ssp}']
    join[f'RSLR1_ssp{ssp}'] = join[f'RSLR1_ssp{ssp}'] & ~(join[f'RSLR2_ssp{ssp}'] | join[f'RSLR3_ssp{ssp}'])

join.to_csv(data_dir.joinpath(f'{source}_mangroves_2020_global_rslr_all_ssps_2020-2100.csv'), index=False)